# Getting Started

In [1]:
import QMzyme
from importlib_resources import files, as_file

## Step 1. QMzyme object instance creation

In [2]:
pdb_file = str(files('QMzyme.data').joinpath('1oh0_equ_from_amber_sim.pdb'))
ksi = QMzyme.GenerateModel(pdb_file, id='1oh0_crystal')

## Step 2. Catalytic center definition

In [3]:
ksi.set_catalytic_center(resname='EQU', resnumber=263) 

## Step 3. Residue selection

#### You must define some cutoff distance in Å. Any residue with at least one atom within this distance of at least one atom of the catalytic center will be included in the subsystem.

In [4]:
ksi.within_distance(distance_cutoff=5)

Model 1 created by the within_distance method has been stored to <QMzyme Structure id=1oh0_crystal>.


## Step 4. Model truncation

In [5]:
ksi.truncate()

## Step 5. QM input generation

In [6]:
ksi.calculateQM(model=None, functional='wb97xd', 
                basis_set='6-31g(d)', opt=True, freq=True, freeze_atoms = ['CA'], 
                charge = -2, mult = 1, mem='32GB', 
                nprocs=16, program='gaussian')

File 1oh0_crystal_1.pdb created.
AQME v 1.5.2 2024/04/16 12:44:03 
Citation: AQME v 1.5.2, Alegre-Requena, J. V.; Sowndarya, S.; Perez-Soto, R.; Alturaifi, T.; Paton, R. AQME: Automated Quantum Mechanical Environments for Researchers and Educators. Wiley Interdiscip. Rev. Comput. Mol. Sci. 2023, DOI: 10.1002/wcms.1663.


o  1oh0_crystal_1 successfully processed at /Users/hrk/git/QMzyme/Tutorials/QCALC


Time QPREP: 0.11 seconds




## Step 6. Store information in JSON file

In [7]:
ksi.write_json()

In [9]:
import json 
with open(f'{ksi.id}.json') as j:
    model_info = json.load(j)

In [11]:
ksi.__dict__

{'level': 'S',
 '_id': '1oh0_crystal',
 'full_id': None,
 'parent': None,
 'child_list': [<Model id=1>],
 'child_dict': {1: <Model id=1>},
 'header': {'name': '',
  'head': '',
  'idcode': '',
  'deposition_date': '1909-01-08',
  'release_date': '1909-01-08',
  'structure_method': 'unknown',
  'resolution': None,
  'structure_reference': [],
  'journal_reference': '',
  'author': '',
  'compound': {'1': {'misc': ''}},
  'source': {'1': {'misc': ''}},
  'has_missing_residues': False,
  'missing_residues': [],
  'biomoltrans': {}},
 'pdb_file': '/Users/hrk/git/QMzyme/QMzyme/data/1oh0_equ_from_amber_sim.pdb',
 'base': <Model id=0>,
 'models': [<Model id=1>],
 'QMzyme_details': {'__init__': {'Time': 'placeholder'}},
 'catalytic_center': [<Residue EQU resseq=263 chain=A]}

In [10]:
model_info["structure"]

KeyError: 'structure'

##### You will see there is a section "QMzyme 1" with information on how the subsystem was selected and what residues comprise the truncated subsystem. If you were to run model.subsystem() again, it would create a new entry under "QMyme 2". 

In [13]:
model_info["QMzyme 1"]["Subsystem selection"]

{'Number of atoms': 406,
 'Distance cutoff': 5,
 'Output file': '1oh0_equ_xstal_subsystem_distance_cutoff5.pdb'}

In [14]:
model_info["QMzyme 1"]["Truncated subsystem"]

{'Number of atoms': 368,
 'Distance cutoff': 5,
 'Residues': [{'Residue name': 'TYR', 'Residue number': 16, 'Chain': 'A'},
  {'Residue name': 'ILE', 'Residue number': 17, 'Chain': 'A'},
  {'Residue name': 'VAL', 'Residue number': 20, 'Chain': 'A'},
  {'Residue name': 'ASP', 'Residue number': 40, 'Chain': 'A'},
  {'Residue name': 'PRO', 'Residue number': 41, 'Chain': 'A'},
  {'Residue name': 'TYR', 'Residue number': 57, 'Chain': 'A'},
  {'Residue name': 'GLY', 'Residue number': 60, 'Chain': 'A'},
  {'Residue name': 'LEU', 'Residue number': 61, 'Chain': 'A'},
  {'Residue name': 'VAL', 'Residue number': 66, 'Chain': 'A'},
  {'Residue name': 'ALA', 'Residue number': 68, 'Chain': 'A'},
  {'Residue name': 'MET', 'Residue number': 84, 'Chain': 'A'},
  {'Residue name': 'PHE', 'Residue number': 86, 'Chain': 'A'},
  {'Residue name': 'VAL', 'Residue number': 88, 'Chain': 'A'},
  {'Residue name': 'MET', 'Residue number': 90, 'Chain': 'A'},
  {'Residue name': 'LEU', 'Residue number': 99, 'Chain': '